In [35]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.utils import load_img
from tensorflow.keras.utils import img_to_array
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import cv2
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from keras.callbacks import LearningRateScheduler
from keras.layers import Dropout
from keras.callbacks import Callback
from tensorflow.keras import layers, models
import tensorflow as tf

In [36]:
# 图片路径
image_folder = "E:\shujuji\metamaterialpicture900features"
# Excel文件路径
excel_path = "E:\\1-1000-90.xlsx"

In [37]:
df = pd.read_excel(excel_path, usecols=[0, 16])  # 使用第1列和第17列的数据，从0开始计数，并跳过第一行

In [38]:
# 初始化图像和性能数组
X = []
y = []

In [39]:
for i in range(1, 651):
    img_path = os.path.join(image_folder, f"lcy{i}.jpg")
    if i in df["编号"].values:
        img = load_img(img_path, target_size=(20, 20))  # 调整为合适的图像大小
        img_array = img_to_array(img)
        img_array = cv2.cvtColor(img_array.astype('uint8'), cv2.COLOR_RGB2GRAY)
        X.append(img_array)
y=df["d33"]

In [40]:
# 转换为NumPy数组
X = np.array(X)
y = np.array(y)

In [41]:
# 图片路径
image_folder1 = "E:\metamaterialpicture\metamaterialpicture900features"
# Excel文件路径
excel_path1 = "E:\metamaterialpicture\metamaterialpicture90\\1-430-90.xlsx"
df1 = pd.read_excel(excel_path1, usecols=[0, 13])  # 使用第1列和第17列的数据，从0开始计数，并跳过第一行
# 初始化图像和性能数组
X1 = []
y1 = []
for i in range(1, 431):
    img_path = os.path.join(image_folder1, f"lcy{i}.jpg")
    if i in df1["编号"].values:
        img = load_img(img_path, target_size=(20, 20))  # 调整为合适的图像大小
        img_array = img_to_array(img)
        img_array = cv2.cvtColor(img_array.astype('uint8'), cv2.COLOR_RGB2GRAY)
        X1.append(img_array)

y1=df1["d33"]
# 转换为NumPy数组
X1 = np.array(X1)
y1 = np.array(y1)

In [42]:
# 图片路径
image_folder2 = "E:\metamaterialpicture\me90features"
# Excel文件路径
excel_path2 = "E:\metamaterialpicture\me90\\1-300-90.xlsx"
df2 = pd.read_excel(excel_path2, usecols=[0, 14])  # 使用第1列和第17列的数据，从0开始计数，并跳过第一行
# 初始化图像和性能数组
X2 = []
y2 = []
for i in range(1, 302):
    img_path = os.path.join(image_folder2, f"lcy{i}.jpg")
    if i in df2["编号"].values:
        img = load_img(img_path, target_size=(20, 20))  # 调整为合适的图像大小
        img_array = img_to_array(img)
        img_array = cv2.cvtColor(img_array.astype('uint8'), cv2.COLOR_RGB2GRAY)
        X2.append(img_array)

y2=df2["d33"]
# 转换为NumPy数组
X2 = np.array(X2)
y2 = np.array(y2)

In [43]:
# 划分训练集和测试集
X3=np.concatenate((X, X1), axis=0)
y3=np.concatenate((y, y1), axis=0)
X4=np.concatenate((X3, X2), axis=0)
y4=np.concatenate((y3, y2), axis=0)
X_train, X_test, y_train, y_test = train_test_split(X4, y4, test_size=0.2, random_state=500)
X_train = np.expand_dims(X_train, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)
X_train = X_train / 255.0
X_test = X_test / 255.0
X_train = X_train.astype('float32')
y_train = y_train.astype('float32')
X_test = X_test.astype('float32')
y_test = y_test.astype('float32')
print(X_train.shape)

(1056, 20, 20, 1)


In [44]:
# 定义ResNet的基本块
class BasicBlock(tf.keras.layers.Layer):
    def __init__(self, filters, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = layers.Conv2D(filters, kernel_size=3, strides=stride, padding='same', use_bias=False)
        self.bn1 = layers.BatchNormalization()
        self.relu = layers.Activation('relu')
        self.conv2 = layers.Conv2D(filters, kernel_size=3, strides=1, padding='same', use_bias=False)
        self.bn2 = layers.BatchNormalization()
        self.downsample = None
        if stride != 1:
            self.downsample = models.Sequential([
                layers.Conv2D(filters, kernel_size=1, strides=stride, use_bias=False),
                layers.BatchNormalization()
            ])

    def call(self, inputs, training=False):
        identity = inputs
        x = self.conv1(inputs)
        x = self.bn1(x, training=training)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x, training=training)
        if self.downsample is not None:
            identity = self.downsample(inputs)
        x += identity
        x = self.relu(x)
        return x

# 定义ResNet模型
class ResNet(tf.keras.Model):
    def __init__(self, block, layer, num_classes=1):
        super(ResNet, self).__init__()
        self.in_channels = 64
        self.conv1 = layers.Conv2D(64, kernel_size=7, strides=2, padding='same', use_bias=False, input_shape=(20,20,1))
        self.bn1 = layers.BatchNormalization()
        self.relu = layers.Activation('relu')
        self.Averagepool = layers.AveragePooling2D(pool_size=(3, 3), strides=2, padding='same')
        self.layer1 = self.build_layer(block, 64, layer[0])
        self.layer2 = self.build_layer(block, 128, layer[1], stride=2)
        self.layer3 = self.build_layer(block, 256, layer[2], stride=2)
        self.layer4 = self.build_layer(block, 512, layer[3], stride=2)
        self.avgpool = layers.GlobalAveragePooling2D()
        self.fc = layers.Dense(num_classes, activation='linear')  # 线性激活函数，用于回归问题

    def build_layer(self, block, filters, blocks, stride=1):
        res_blocks = models.Sequential()
        res_blocks.add(block(filters, stride=stride))
        for _ in range(1, blocks):
            res_blocks.add(block(filters, stride=1))
        return res_blocks

    def call(self, inputs, training=False):
        x = self.conv1(inputs)
        x = self.bn1(x, training=training)
        x = self.relu(x)
        x = self.avgpool(x)

        x = self.layer1(x, training=training)
        x = self.layer2(x, training=training)
        x = self.layer3(x, training=training)
        x = self.layer4(x, training=training)

        x = self.avgpool(x)
        x = self.fc(x)
        return x

# 创建ResNet-18模型
resnet18 = ResNet(BasicBlock, [2, 2, 2, 2], num_classes=1)

# 编译模型，选择合适的优化器和损失函数
resnet18.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])

In [45]:
# 在 model.fit 中使用 callbacks 参数将 LearningRateScheduler 加入训练过程
history = resnet18.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_test, y_test))

Epoch 1/50


ValueError: in user code:

    File "D:\anaconda3\envs\tf2\lib\site-packages\keras\engine\training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "D:\anaconda3\envs\tf2\lib\site-packages\keras\engine\training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "D:\anaconda3\envs\tf2\lib\site-packages\keras\engine\training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "D:\anaconda3\envs\tf2\lib\site-packages\keras\engine\training.py", line 1023, in train_step
        y_pred = self(x, training=True)
    File "D:\anaconda3\envs\tf2\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\Administrator\AppData\Local\Temp\__autograph_generated_file9d2pau73.py", line 14, in tf__call
        x = ag__.converted_call(ag__.ld(self).layer1, (ag__.ld(x),), dict(t

In [ ]:
plt.plot(history.history['loss'][5:], label='Training Loss')
plt.plot(history.history['val_loss'][5:], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# 在训练集上计算 R² 分数
y_train_pred = resnet18.predict(X_train)
r2_train = r2_score(y_train, y_train_pred)

# 在验证集上计算 R² 分数
y_test_pred = resnet18.predict(X_test)
r2_test = r2_score(y_test, y_test_pred)

# 打印 R² 分数
print(f'R² 分数 - 训练集: {r2_train:.4f}')
print(f'R² 分数 - 验证集: {r2_test:.4f}')